# Salarios

## 1. importar las librerías + csv con los datos de la encuesta. 

In [1]:
import pandas as pd
import plotly.express as px  
from dash import Dash, dcc, html, Input, Output
import numpy as np
import PyCurrency_Converter

#slider
import plotly.graph_objects as go
from ipywidgets import interact


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv ('../data/survey_results_public2021.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta
 #mostrar df ()
df20 = pd.read_csv ('../data/survey_results_public2020.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta
 #mostrar df ()

# fuente: https://stats.bis.org/statx/srs/table/I3?c=&p=202204&m=A
df_currency = pd.read_csv ('../data/conversor.csv')


## 2. Preprocesar datos.

Tratar las columnas/conjunto de datos para comenzar a crear los gráficos. En este caso Age1stcode

In [4]:

# nombre columna: lo que han preguntado en el survey.
#currency: Which currency do you use day-to-day?
#comptotal: What is your current total compensation (answered in currency from currency column)
#compfreq: Is that compensation weekly, monthly, or yearly?

#copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
df=df_csv[['DevType','Currency', 'CompTotal', 'CompFreq']].dropna().copy()

df= df.drop(df[df.Currency == "none"].index) #eliminar todas las none currency

# 1. convertir todas las currency a la misma moneda. X ej dolares

#convertir la moneda a un string de 3 letras.
df['Currency']= df.Currency.str.split().str.get(0)

#unir ambos df 
df3 = pd.merge(df_currency, df, how='inner')

#sustituir la compensacion en la moneda del pais por su equivalente en USD
df3["CompTotal"] = (df3["CompTotal"]/df3["Conversion"]).round(2) #redondear el numero a 2 decmales

df3

KeyError: "['Currency'] not in index"

In [3]:
#ahora se debe normalizar la compensacion. Para ello se asumira que: 
# un mes tiene 4 seamanas y un anyo tiene 12 meses. 
# un anyo tiene tambien 52 semanas

condiciones = [(df3['CompFreq'] =='Monthly'), (df3['CompFreq'] =='Weekly'), (df3['CompFreq'] =='Yearly')]


In [4]:
df3['semana'] = np.select(condiciones, [(df3['CompTotal'])/4, df3['CompTotal'], (df3['CompTotal'])/52]).round(2)      
df3['mes'] = np.select(condiciones, [df3['CompTotal'], (df3['CompTotal'])*4, (df3['CompTotal'])/12]).round(2)      
df3['anyo'] = np.select(condiciones, [(df3['CompTotal'])*12, (df3['CompTotal'])*52, df3['CompTotal']]).round(2) 
df3

,Coutry,Currency,Conversion,DevType,CompTotal,CompFreq,semana,mes,anyo
0,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",820.17,Monthly,205.04,820.17,9842.04
1,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",84.62,Weekly,84.62,338.48,4400.24
2,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",260.37,Monthly,65.09,260.37,3124.44
3,Afghanistan,AFN,76.8135,"Developer, mobile;Developer, front-end;Develop...",455.65,Monthly,113.91,455.65,5467.80
4,Afghanistan,AFN,76.8135,System administrator,13018.54,Monthly,3254.64,13018.54,156222.48
...,...,...,...,...,...,...,...,...,...
292977,Vietnam,VND,23152.9000,"Developer, mobile;Developer, desktop or enterp...",3455.29,Monthly,863.82,3455.29,41463.48
292978,Vietnam,VND,23152.9000,"Developer, back-end",993.40,Monthly,248.35,993.40,11920.80
292979,Yemen,YER,1240.7580,"Developer, mobile;Developer, front-end;Develop...",322.38,Monthly,80.60,322.38,3868.56
292980,Yemen,YER,1240.7580,"Developer, mobile;Developer, full-stack;Studen...",80.60,Monthly,20.15,80.60,967.20


# hay una columna con la conversion hecha. 

Rehacer el dataframe.

In [2]:
#df4=df_csv[['DevType','ConvertedCompYearly', 'YearsCodePro', 'Age1stCode']].dropna().copy()
df4=df_csv[['DevType','ConvertedComp', 'YearsCodePro', 'Age1stCode']].dropna().copy()

df4

,DevType,ConvertedComp,YearsCodePro,Age1stCode
Respondent,,,,
8,"Developer, back-end;Developer, desktop or ente...",116000.0,13,12
10,"Database administrator;Developer, full-stack;D...",32315.0,4,14
11,"Developer, back-end;Developer, desktop or ente...",40070.0,2,13
12,"Designer;Developer, front-end",14268.0,7,42
13,"Designer;Developer, back-end",38916.0,20,14
...,...,...,...,...
65619,"Academic researcher;Developer, back-end;Educator",984.0,2,19
65625,"Developer, QA or test",19428.0,5,17
65629,"Developer, back-end;Developer, front-end;Devel...",200.0,20,15


In [3]:

df4.dtypes

DevType           object
ConvertedComp    float64
YearsCodePro       int64
Age1stCode         int64
dtype: object

In [4]:
#separar las filas que tienen mas de 1 devtype
#df4['DevType'] = df4['DevType'].str.split(';').apply(lambda x: [e.strip() for e in x]).tolist()

#agrupar por tipo de dev



#df4=df4.explode('DevType').groupby('DevType').agg(['median'])
df4=df4.assign(DevType=df4['DevType'].str.split(';')).explode('DevType').groupby('DevType').agg(avg_pro=('YearsCodePro', 'mean'),
                                                  avg_age=('Age1stCode','mean'),
                                                  mean_money=('ConvertedComp','median'),
                                                 respuestas=('DevType','count')).reset_index()
#eliminar informacion irrelevante
df4.drop(df4.index[df4['DevType'] == "Other (please specify):"], inplace=True)

In [2]:
df4=df21[['DevType','ConvertedCompYearly', 'YearsCodePro','Age1stCode']].dropna().copy()
    
    # eliminar filas en anyos que tengan texto. 
df4.drop(df4[df4['YearsCodePro'] == "Less than 1 year"].index, inplace = True) 
df4.drop(df4[df4['YearsCodePro'] == "More than 50 years"].index, inplace = True)

df4.drop(df4[df4['Age1stCode'] == "Younger than 5 years"].index, inplace = True) 
df4.drop(df4[df4['Age1stCode'] == "Older than 85"].index, inplace = True)

#convertir columnas a entero
#df4["Age1stCode"] = pd.to_numeric(df4["Age1stCode"])
df4["YearsCodePro"] = pd.to_numeric(df4["YearsCodePro"])

df21.Age1stCode.value_counts()




In [3]:
#separar filas con mas de un devtype y despues agruparlas
df4=df4.assign(DevType=df4['DevType']
               .str.split(';')).explode('DevType').groupby('DevType').agg(avg_pro=('YearsCodePro', 'mean'), #nueva columna que calculara la media de yearscodepro
             avg_age=('Age1stCode','mean'), 
             median_money=('ConvertedCompYearly','median'), #nueva columna con mediana del salario
             # contar los diferentes tipos de devtype                                                                                    
             respuestas=('DevType','count')).reset_index() #reiniciar el indice

df4

,DevType,avg_pro,avg_age,median_money,respuestas
0,Academic researcher,8.760882,18.924816,51300.0,1769
1,Data or business analyst,11.178661,19.155230,62598.5,2390
2,Data scientist or machine learning specialist,8.570110,18.631523,64859.0,2817
3,Database administrator,11.970092,17.929703,56344.0,3912
4,Designer,11.883285,18.539986,52074.0,2776
5,DevOps specialist,11.024925,17.747751,71046.5,5336
6,"Developer, QA or test",10.285771,18.106815,57480.0,2509
7,"Developer, back-end",9.865631,18.245173,58368.0,20667
8,"Developer, desktop or enterprise applications",11.898223,18.222106,59676.0,7654
9,"Developer, embedded applications or devices",11.365981,17.913810,59454.0,3063


In [2]:
#retocada x2
def salario (df,anyo):
    
    #para poder reutilizar la columna en el df de 2021 y 2020 sin hacer grandes cambios, hay que renombrar ConvertedComp
    df.rename(columns = {'ConvertedComp':'ConvertedCompYearly'}, inplace = True)
    
    #copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
    df4=df[['DevType','ConvertedCompYearly', 'YearsCodePro','Age1stCode']].dropna().copy()
    
    if (anyo == 21): #Age1stCode en el df del 2021 son string en lugar de enteros.
         
        #crear diccionario con los cambios de valores
        edadesTransformadas = {"Younger than 5 years" : 4, "5 - 10 years" : 7, "11 - 17 years": 18, "18 - 24 years" : 21,
        "25 - 34 years" : 30, "35 - 44 years" : 40, "45 - 54 years": 50, "55 - 64 years" : 60, 
        "Older than 64 years": 70}
        
        #cambiar los valores.
        df4=df4.replace({"Age1stCode": edadesTransformadas})
    
    #resto es igual para ambos df.
    
    # eliminar filas en anyos que tengan texto. 
    df4.drop(df4[df4['YearsCodePro'] == "Less than 1 year"].index, inplace = True) 
    df4.drop(df4[df4['YearsCodePro'] == "More than 50 years"].index, inplace = True)
    
    df4.drop(df4[df4['Age1stCode'] == "Younger than 5 years"].index, inplace = True) 
    df4.drop(df4[df4['Age1stCode'] == "Older than 85"].index, inplace = True)
    
    #convertir columnas a entero
    df4["Age1stCode"] = pd.to_numeric(df4["Age1stCode"])
    df4["YearsCodePro"] = pd.to_numeric(df4["YearsCodePro"])
    
    #separar filas con mas de un devtype y despues agruparlas
    df4=df4.assign(DevType=df4['DevType'].str.split(';')).explode('DevType').groupby('DevType').agg(avg_pro=('YearsCodePro', 'mean'), #nueva columna que calculara la media de yearscodepro
                 avg_age=('Age1stCode','mean'), 
                 median_money=('ConvertedCompYearly','median'), #nueva columna con mediana del salario
                 # contar los diferentes tipos de devtype                                                                                    
                 respuestas=('DevType','count')).reset_index() #reiniciar el indice

    #eliminar informacion irrelevante
    df4.drop(df4.index[df4['DevType'] == "Other (please specify):"], inplace=True)
    
    return df4

Para lidiar con rangos de edades, algunos de los cuales tienen texto, se va a calcular una nueva columna con la media de todos ellos. 


## 3. Grafico. 

En este caso, un diagrama de barras.

In [4]:
app = Dash(__name__)
server = app.server #heroku
app.layout = html.Div([

    html.H1("Tipo de desarrollador", style={'text-align': 'center'}), #cabecero h1. Header
    
    #primera mini prueba con un menu desplegable.
    dcc.Dropdown(id="opt4",  
                 options=[ #el usuario va a ver las label.
                     {"label": "2020", "value": 2020},
                     {"label": "2021", "value": 2021}],
                 multi=False,
                 value=2020,
                 style={'width': "40%"}
                 ),

    dcc.Graph(id='my_survey', figure={}) # graph container

])

In [5]:
@app.callback(
    Output(component_id='my_survey', component_property='figure'),
    Input(component_id='opt4', component_property='value'))
def update_graph(opt4):
    df = pd.DataFrame()
    anyo = 0

    if (opt4 == 2020): 
        df= df20 
        anyo = 20
    else: 
        df = df21 
        anyo = 21

    fig =  px.scatter(salario (df,anyo), x= "avg_pro", y= "avg_age", size="median_money", 
                      symbol="DevType", color="respuestas", hover_name="DevType")

    #quitar color y grid del grafico
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente

    return fig

## 4. run server

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
